In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class State:
    def __init__(self, state, parent, move, depth, cost, key):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.key = key
        if self.state:
            self.map = ''.join(str(e) for e in self.state)
    def __eq__(self, other):
        return self.map == other.map
    def __lt__(self, other):
        return self.map < other.map

In [0]:
from math import sqrt
from collections import deque
from heapq import heappush, heappop
import time
# Stopping Condition and the goal state
goalState = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,0]
goalNode = State
initialState = []   
puzzleLen = 0       
puzzleSide = 0      
nodesExpanded = 0   
maxDepthReached = 0 
maxFringeSize = 0   
moves = []          


## Breadth first search (BFS)

In [0]:
def bfs(startState):
    global goalNode, maxFringeSize, maxDepthReached
    visited, queue = set(), deque([State(startState, None, None, 0, 0, 0)])
    while queue:                            
        node = queue.popleft()              
        visited.add(node.map)               
        if node.state == goalState:
            goalNode = node
            return True, queue
        childNodes = expand(node)
        for child in childNodes:            
            if child.map not in visited:    
                queue.append(child)         
                visited.add(child.map)      
                if child.depth > maxDepthReached:
                    maxDepthReached += 1
        if len(queue) > maxFringeSize:
            maxFringeSize = len(queue)
    return False, None    

## Depth first Search (DFS)

In [0]:
def dfs(startState):
    global goalNode, maxFringeSize, maxDepthReached
    visited, stack = set(), list([State(startState, None, None, 0, 0, 0)])
    while stack:                                
        node = stack.pop()                      
        visited.add(node.map)                  
        if node.state == goalState:
            goalNode = node
            return True, stack
        neighbors = reversed(expand(node))     
        for neighbor in neighbors:              
            if neighbor.map not in visited:     
                stack.append(neighbor)          
                visited.add(neighbor.map)       
                if neighbor.depth > maxDepthReached:
                    maxDepthReached += 1
        if len(stack) > maxFringeSize:
            maxFringeSize = len(stack)      
    return False, None

## Greddy Search

In [0]:
def greedy(startState, heuristicFunc):
    global goalNode, maxFringeSize, maxDepthReached
    visited, pQueue = set(), list()
    key = heuristicFunc(startState)
    root = State(startState, None, None, 0, 0, key)
    entry = (key, 0, root)
    heappush(pQueue, entry)
    while pQueue:
        node = heappop(pQueue)
        #print(node)
        visited.add(node[2].map)
        if node[2].state == goalState:
            goalNode = node[2]
            return True, pQueue
        neighbors = expand(node[2])
        for neighbor in neighbors:
            neighbor.key = heuristicFunc(neighbor.state)
            entry = (neighbor.key, neighbor.move, neighbor)
            if neighbor.map not in visited:
                heappush(pQueue, entry)
                visited.add(neighbor.map)
                if neighbor.depth > maxDepthReached:
                    maxDepthReached += 1                    
        if len(pQueue) > maxFringeSize:
            maxFringeSize = len(pQueue)   
    return False, None

## A* Search 

In [0]:
def ast(startState, heuristicFunc):
    global goalNode, maxFringeSize, maxDepthReached
    visited, pQueue = set(), list()
    key = heuristicFunc(startState)
    root = State(startState, None, None, 0, 0, key)
    entry = (key, 0, root)
    heappush(pQueue, entry)
    while pQueue:
        node = heappop(pQueue)
        visited.add(node[2].map)
        if node[2].state == goalState:
            goalNode = node[2]
            return True, pQueue
        neighbors = expand(node[2])
        for neighbor in neighbors:
            neighbor.key = neighbor.cost + heuristicFunc(neighbor.state)
            entry = (neighbor.key, neighbor.move, neighbor)
            if neighbor.map not in visited:
                heappush(pQueue, entry)
                visited.add(neighbor.map)
                if neighbor.depth > maxDepthReached:
                    maxDepthReached += 1                    
        if len(pQueue) > maxFringeSize:
            maxFringeSize = len(pQueue)
    return False, None

## Heuristic Function 1 : number of misplaced tiles

In [0]:
def h1(state):
    count = 0
    for i in range(0,puzzleLen):
        if not (state.index(i) == goalState.index(i)) : 
            count+=1
    return count 

## Heuristic Function 2 : sum of the distances of every tile to its goal position.

In [0]:
def h2(state): 
    return sum(abs(p%puzzleSide - g%puzzleSide) + abs(p//puzzleSide - g//puzzleSide)
               for p,g in ((state.index(i),goalState.index(i)) 
               for i in range(1, puzzleLen))) 


In [0]:
def expand(node):
  # creates valid child nodes
    global nodesExpanded
    nodesExpanded += 1
    childNodes = []
    childNodes.append(State(validMove(node.state,'DOWN'),node,'DOWN',node.depth + 1, node.cost + 1, 0)) 
    childNodes.append(State(validMove(node.state,'LEFT'),node,'LOW',node.depth + 1, node.cost + 1, 0)) 
    childNodes.append(State(validMove(node.state,'RIGHT'),node,'RIGHT',node.depth + 1, node.cost + 1, 0))
    childNodes.append(State(validMove(node.state,'UP'),node,'UP',node.depth + 1, node.cost + 1, 0))
    nodes = [child for child in childNodes if child.state]
    return nodes

In [0]:
def validMove(state, position):
    # Verify next move
    newState = state[:]
    index = newState.index(0) 
    if position == 'UP': 
        if index not in range(0, puzzleSide):           
            temp = newState[index - puzzleSide]
            newState[index - puzzleSide] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'DOWN':        
        if index not in range(puzzleLen - puzzleSide, puzzleLen):
            temp = newState[index + puzzleSide]
            newState[index + puzzleSide] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'LEFT': 
        if index not in range(0, puzzleLen, puzzleSide):
            temp = newState[index - 1]
            newState[index - 1] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'RIGHT': 
        if index not in range(puzzleSide - 1, puzzleLen, puzzleSide):
            temp = newState[index + 1]
            newState[index + 1] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None

In [0]:
def get(dataList):
    global puzzleLen, puzzleSide
    
    data = dataList.split(',') 
    for element in data:
        initialState.append(int(element))
    puzzleLen = len(initialState)                  
    puzzleSide = int(sqrt(puzzleLen))       


In [0]:
def backtrack():
    currentNode = goalNode
    while initialState != currentNode.state : 
        moves.insert(0, currentNode.move)
        currentNode = currentNode.parent
    return moves


In [0]:
def output(fringe, time, testNum):
    if fringe:
        global moves
        moves = backtrack()
        print("\nProblem State : "+str(initialState))
        print("\npath_to_goal : " + str(moves))
        print("\ncost_of_path(Moves): " + str(len(moves)))
        print("\nnodes_expanded: " + str(nodesExpanded))
        print("\nfringe_size: " + str(len(fringe)))
        print("\nmax_fringe_size: " + str(maxFringeSize))
        print("\nsearch_depth: " + str(goalNode.depth))
        print("\nmax_search_depth: " + str(maxDepthReached))
        print("\nrunning_time: " + format(time, '.8f'))
    else:
        print("<-- # UNSOLVABLE # -->")
        print("\nnodes_expanded: " + str(nodesExpanded))
        print("\nmax_fringe_size: " + str(maxFringeSize))
        print("\nmax_search_depth: " + str(maxDepthReached))
        print("\nrunning_time: " + format(time, '.8f'))
      

In [0]:
def main():
    algorithm = input('--> Please select the algorithm \
                      \n1. bfs : Breadth First Search \
                      \n2. dfs : Depth First Search \
                      \n3. ast : A Star Search\
                      \n4. greedy : Greedy Search\
                      \n enter the selection : ')
    if(algorithm == 'greedy' or algorithm == 'ast'):
        heuristic = input('-- Please select the Heuristic Function\
                      \n h1 : number of misplaced tiles\
                      \n h2 : sum of the distances of every tile to its goal position.\
                      \n-- Enter your choice : ')
    
        heuristicFunc = heuristic_map[heuristic]
    else:
        heuristicFunc = None
    function_map = {'bfs': bfs, 'dfs': dfs,'greedy' : greedy,'ast' : ast}
    heuristic_map = {'h1' : h1,'h2' : h2}
    data = input('--> Enter puzzle elements : ')    
    get(data)
    function = function_map[algorithm]  
    start = time.time()       
    search, fringe = function(initialState, heuristicFunc)  
    stop = time.time()
    if not search : 
        output(fringe, stop-start, 'No_Solution')
    else : 
        output(fringe, stop-start, 'h2')        

In [0]:
if __name__ == '__main__':
    main()